In [1]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, kendalltau
import matplotlib.pyplot as plt
import seaborn as sns
import geocoder
%matplotlib inline

In [2]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')

In [3]:
train['num_features'] = train['features'].str.len()
train['num_words_desc'] = train['description'].str.split().str.len()
train["num_photos"] = train["photos"].apply(len)
train["created"] = pd.to_datetime(train["created"])
train["created_year"] = train["created"].dt.year
train["created_month"] = train["created"].dt.month
train["created_day"] = train["created"].dt.day
train["created_hour"] = train["created"].dt.hour
train['low_int'] = (train['interest_level'] == 'low').astype(int)
train['med_int'] = (train['interest_level'] == 'medium').astype(int)
train['high_int'] = (train['interest_level'] == 'high').astype(int)
missingCoords = train[(train.longitude == 0) | (train.latitude == 0)]
missingGeoms = (missingCoords.street_address + ' New York').apply(geocoder.google)
train.loc[(train.longitude == 0) | (train.latitude == 0), 'latitude'] = missingGeoms.apply(lambda x: x.lat)
train.loc[(train.longitude == 0) | (train.latitude == 0), 'longitude'] = missingGeoms.apply(lambda x: x.lng)

In [ ]:
def transformData(dataSet):
    dataSet['num_features'] = dataSet['features'].str.len()
    dataSet['num_words_desc'] = dataSet['description'].str.split().str.len()
    dataSet["num_photos"] = dataSet["photos"].apply(len)
    dataSet["created"] = pd.to_datetime(dataSet["created"])
    dataSet['low_int'] = (dataSet['interest_level'] == 'low').astype(int)
    dataSet['med_int'] = (dataSet['interest_level'] == 'medium').astype(int)
    dataSet['high_int'] = (dataSet['interest_level'] == 'high').astype(int)
    missingCoords = dataSet[(dataSet.longitude == 0) | (dataSet.latitude == 0)]
    missingGeoms = (missingCoords.street_address + ' New York').apply(geocoder.google)
    dataSet.loc[(dataSet.longitude == 0) | (dataSet.latitude == 0), 'latitude'] = missingGeoms.apply(lambda x: x.lat)
    dataSet.loc[(dataSet.longitude == 0) | (dataSet.latitude == 0), 'longitude'] = missingGeoms.apply(lambda x: x.lng)
    #

In [5]:
train['low_int'] = (train['interest_level'] == 'low').astype(int)
train['med_int'] = (train['interest_level'] == 'medium').astype(int)
train['high_int'] = (train['interest_level'] == 'high').astype(int)
missingCoords = train[(train.longitude == 0) | (train.latitude == 0)]
missingGeoms = (missingCoords.street_address + ' New York').apply(geocoder.google)
train.loc[(train.longitude == 0) | (train.latitude == 0), 'latitude'] = missingGeoms.apply(lambda x: x.lat)
train.loc[(train.longitude == 0) | (train.latitude == 0), 'longitude'] = missingGeoms.apply(lambda x: x.lng)

In [6]:
numericalcolumns = ['bathrooms', 'bedrooms', 'price', 'num_features', 'num_words_desc', 'latitude', 'longitude', 'num_photos']

In [7]:
def corrtable(x):
    a = []
    for i in numericalcolumns:
        b = kendalltau(train[x + '_int'],train[i])
        a.append([i, b[0], b[1]])
    a = pd.DataFrame(a)
    a.columns = ['columns', 'correlation', 'pvalue']
    a = a[a['pvalue'] <0.05]
    return a

In [169]:
corrtable('high')

,columns,correlation,pvalue
0,bathrooms,-0.056114,5.124639e-78
2,price,-0.157578,0.000000e+00
3,num_features,-0.025558,1.646384e-17
4,num_words_desc,0.010761,3.359975e-04
7,num_photos,0.019915,3.222530e-11


In [170]:
corrtable('med')

,columns,correlation,pvalue
0,bathrooms,-0.041982,1.814071e-44
1,bedrooms,0.040477,1.849118e-41
2,price,-0.107012,1.757325e-278
3,num_features,0.051508,4.989573e-66
4,num_words_desc,0.071286,1.007247e-124
7,num_photos,0.050666,6.031457e-64


In [171]:
corrtable('low')

,columns,correlation,pvalue
0,bathrooms,0.070851,3.124565e-123
1,bedrooms,-0.040187,6.810998e-41
2,price,0.189057,0.000000e+00
3,num_features,-0.032025,1.388356e-26
4,num_words_desc,-0.071151,2.934715e-124
7,num_photos,-0.057704,2.163549e-82


In [9]:
missingBath = train[train.bathrooms ==0] #most places with missing bathrooms have 1 bathroom; impute or set as 1
missingBed = train[train.bedrooms ==0] #studio apartments; group? 0,1,2,3,4, >=5
missingBuild = train[train.building_id =='0'] 

#can have half bathrooms; round up bathrooms? group?1,2,3, >=4
#change bathrooms and bedrooms to categories?
#group by features that show up more than a certain percentage of the time?

In [173]:
train

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,...,manager_id,photos,price,street_address,num_features,num_words_desc,num_photos,low_int,med_int,high_int
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,...,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,0,93,5,0,1,0
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,...,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue,5,0,11,1,0,0
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,...,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street,4,92,8,0,0,1
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,...,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street,2,78,3,1,0,0
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,...,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street,1,67,3,1,0,0
100014,2.0,4,38a913e46c94a7f46ddf19b756a9640c,2016-04-19 04:24:47,,West 18th Street,[],medium,40.7429,6894514,...,b209e2c4384a64cc307c26759ee0c651,[https://photos.renthop.com/2/6894514_9abb8592...,7995,350 West 18th Street,0,0,5,0,1,0
100016,1.0,2,3ba49a93260ca5df92fde024cb4ca61f,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,"[prewar, elevator, Dogs Allowed, Cats Allowed,...",low,40.8012,6930771,...,01287194f20de51872e81f660def4784,[https://photos.renthop.com/2/6930771_7e3622b6...,3600,210 West 107th Street,8,85,10,1,0,0
100020,2.0,1,0372927bcb6a0949613ef5bf893bbac7,2016-04-13 06:01:42,"This huge sunny ,plenty of lights 1 bed/2 bath...",West 21st Street,"[Doorman, Elevator, Pre-War, Terrace, Laundry ...",low,40.7427,6867392,...,e6472c7237327dd3903b3d6f6a94515a,[https://photos.renthop.com/2/6867392_b18283f6...,5645,155 West 21st Street,8,130,5,1,0,0
100026,1.0,1,a7efbeb58190aa267b4a9121cd0c88c0,2016-04-20 02:36:35,<p><a website_redacted,Hamilton Terrace,"[Cats Allowed, Dogs Allowed, Elevator, Laundry...",medium,40.8234,6898799,...,c1a6598437b7db560cde66e5a297a53f,[https://photos.renthop.com/2/6898799_3759be4c...,1725,63 Hamilton Terrace,4,2,5,0,1,0
100027,2.0,4,0,2016-04-02 02:58:15,This is a spacious four bedroom with every bed...,522 E 11th,"[Dishwasher, Hardwood Floors]",low,40.7278,6814332,...,23a01ea7717b38875f5b070282d1b9d2,[https://photos.renthop.com/2/6814332_e19a8552...,5800,522 E 11th,2,160,9,1,0,0


In [10]:
sum(train.num_photos)

276714

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_SLICEMAP',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_prepare__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__idiv__',
 '__imul__',
 '__init__',
 '__int__',
 '__invert__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__'